В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [79]:
!pip install opendatasets --upgrade --quiet

In [80]:
# import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import opendatasets as od
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

%load_ext autoreload
%autoreload 2

from process_bank_churn import preprocess_data, preprocess_new_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
# load dataset from Kaggle
dataset_url = 'https://www.kaggle.com/competitions/bank-customer-churn-prediction-dlu/data'
od.download(dataset_url)
data_dir = './bank-customer-churn-prediction-dlu/'
os.listdir(data_dir)
raw_df_url = '{}train.csv'.format(data_dir)
print(raw_df_url)

Skipping, found downloaded files in "./bank-customer-churn-prediction-dlu" (use force=True to force download)
./bank-customer-churn-prediction-dlu/train.csv


In [82]:
raw_df = pd.read_csv(raw_df_url)
data, scaler, encoder, input_cols = preprocess_data(raw_df, False, ['id', 'CustomerId', 'Surname'])
train_X = data['train_X']
train_y = data['train_y'].to_numpy()
val_X = data['val_X']
val_y = data['val_y'].to_numpy()

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [83]:
def predict_and_auc(train_X, train_y, val_X, val_y, model):
  y_train_proba = model.predict_proba(train_X)[:,1]
  auc_train = roc_auc_score(train_y, y_train_proba)
  y_val_proba = model.predict_proba(val_X)[:,1]
  auc_val = roc_auc_score(val_y, y_val_proba)
  print('AUC score на тестовій вибірці дорівнює {x: .4f}'.format(x = auc_train))
  print('AUC score на валідаційній вибірці дорівнює {x: .4f}'.format(x = auc_val))
  return auc_train, auc_val

In [84]:
knn = KNeighborsClassifier()
knn.fit(train_X, train_y)
knn.predict_proba(train_X)[:,1]

array([0.2, 0.6, 0.6, ..., 0.4, 0. , 0.2])

In [85]:
auc_train, auc_val = predict_and_auc(train_X, train_y, val_X, val_y, knn)

AUC score на тестовій вибірці дорівнює  0.8228
AUC score на валідаційній вибірці дорівнює  0.5794


**Висновки**

1. Модель погана, присутній overfit (high variance), тому що AUC на тестовій вибірці високий (0.8228), а на валідаційній - дуже низький (0.5794). Це означає, що модель підігналась під тестові дані та погано генералізує.

1. AUC score на валідаційній вибірці дорівнює 0.5794, що дуже близько до 0.5. Це означає, що наша модель тільки незначно краще, ніж випадкове вгадування

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [86]:
knn = KNeighborsClassifier()
knn_params = {'n_neighbors': np.arange(1, 25,1)}
knn_gs = GridSearchCV(knn, knn_params, cv = 5, scoring = 'roc_auc')
knn_gs.fit(train_X, train_y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             scoring='roc_auc')

In [87]:
knn_gs.best_params_

{'n_neighbors': 24}

In [88]:
knn_gs.best_score_

0.5908338758543771

In [89]:
knn_best = knn_gs.best_estimator_

In [90]:
auc_train, auc_val = predict_and_auc(train_X, train_y, val_X, val_y, knn_best)

AUC score на тестовій вибірці дорівнює  0.6964
AUC score на валідаційній вибірці дорівнює  0.5886


**Висновки**

1. У знайденому естиматорі немає overfit, але є underfit (high bias). Це можна припустити, тому що auc score як на тренувальному, так і на валідаційному сеті мають низьке значення.
2. Модель незначно покращилась у порівнянні з попередньою (auc score 0,5886 проти 0,5794). Але вона суттєво гірше, ніж модель DesicionTreeClassifier, знайдена у попередньому ДЗ (auc score на тестовому сеті 0,9243).

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_seed`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [91]:
%%time
dt = DecisionTreeClassifier(random_state = 12)
dt_params = {'max_depth': np.arange(1, 20,2), 'max_leaf_nodes': np.arange(2,10,1)}
dt_gs = GridSearchCV(dt, dt_params, cv = 3, scoring = 'roc_auc')
dt_gs.fit(train_X, train_y)
auc_train, auc_val = predict_and_auc(train_X, train_y, val_X, val_y, knn_best)

AUC score на тестовій вибірці дорівнює  0.6964
AUC score на валідаційній вибірці дорівнює  0.5886
CPU times: user 5.51 s, sys: 28.5 ms, total: 5.54 s
Wall time: 5.56 s


In [92]:
dt_gs.best_score_

0.8982564999514152

In [93]:
dt_best = dt_gs.best_estimator_

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [94]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [95]:
%%time
dt = DecisionTreeClassifier(random_state = 12)
dt_rs = RandomizedSearchCV(dt, params_dt, n_iter = 100,  verbose = 0, refit = True, cv = 3, scoring = 'roc_auc')
dt_rs.fit(train_X, train_y)

CPU times: user 5.45 s, sys: 4.11 ms, total: 5.45 s
Wall time: 5.54 s


RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=12),
                   n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   scoring='roc_auc')

In [101]:
dt_best = dt_rs.best_estimator_
dt_best.fit(train_X, train_y)

DecisionTreeClassifier(criterion='entropy', max_depth=13, max_leaf_nodes=14,
                       min_samples_leaf=4, random_state=12)

In [102]:
auc_train, auc_val = predict_and_auc(train_X, train_y, val_X, val_y, dt_best)

AUC score на тестовій вибірці дорівнює  0.9160
AUC score на валідаційній вибірці дорівнює  0.9155


**Висновки**

1. Модель має суттєво кращі показники, ніж попередні. Високий рівень auroc на тестовій та валідаційній вибірках свідчать про те, що модель добре генералізуєу

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [103]:
test_raw_df = pd.read_csv('{}/test.csv'.format(data_dir))
test_raw_df['Exited'] = -1
test_data = preprocess_new_data(test_raw_df, scaler, encoder, False, ['id', 'CustomerId', 'Surname'])

test_raw_df['Exited'] = dt_best.predict_proba(test_data['new_X'])[:,1]
submission_df = test_raw_df[['id','Exited']].copy()
submission_df.to_csv('submission.csv', index = False)